In [4]:
import polars as pl
import pandas as pd

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)

In [5]:
lf = pl.scan_csv('/data/*.csv')

In [6]:
lf.describe()

statistic,mmsi,time,lat,lon,speed,course,heading,vessel_name,imo
str,f64,str,f64,f64,f64,f64,f64,str,f64
"""count""",4.50860784e8,"""450860784""",4.50860784e8,4.50860784e8,4.50860784e8,4.48550133e8,3.3731571e8,"""450826196""",4.18743515e8
"""null_count""",0.0,"""0""",0.0,0.0,0.0,2.310651e6,1.13545074e8,"""34588""",3.2117269e7
"""mean""",3.7987e8,"""2015-07-13 05:43:54.063985""",33.752925,-93.346497,5.190558,183.806022,183.107138,null,2.4358e7
"""std""",1.2985e8,null,10.348719,26.183735,6.738095,104.837372,103.898169,null,9.5470e7
"""min""",99510.0,"""2015-01-01 00:00:00""",9.95039,-179.99931,0.0,0.0,0.0,null,1.0
"""25%""",3.38564e8,"""2015-03-13 19:10:38""",28.45999,-116.83153,0.0,99.2,95.0,null,8.428583e6
"""50%""",3.6701871e8,"""2015-06-11 03:34:38""",30.37519,-89.03003,0.2,188.3,184.0,null,9.217888e6
"""75%""",3.72081e8,"""2015-10-07 03:00:25""",40.63648,-76.97082,11.0,272.5,273.0,null,9.350379e6
"""max""",8.6675523e8,"""2016-10-31 23:59:59""",89.4922,148.32682,102.3,359.9,510.0,null,9.264776e8


In [12]:
monthly_df = (
    lf
    .with_columns(
        month = pl.col('time').dt.month_start().dt.date()
    )
    .group_by('month')
    .agg(
        imo_count = pl.col('imo').n_unique(),
        vessel_count = pl.col('mmsi').n_unique(),
        pings_per_vessel = pl.col('mmsi').count()/pl.col('mmsi').n_unique(),

    )
    .sort('month')
    .collect()
)

monthly_df.head(10)

month,imo_count,vessel_count,pings_per_vessel
date,u32,u32,f64
2015-01-01,356,367,134487.678474
2015-02-01,321,330,127603.190909
2015-03-01,335,347,138855.389049
2015-04-01,324,335,116121.023881
2015-05-01,341,352,104569.721591
2015-06-01,291,302,108484.430464
2015-07-01,297,311,105644.758842
2015-08-01,290,303,98590.940594
2015-09-01,277,288,76900.006944


In [13]:
monthly_df.describe()

statistic,month,imo_count,vessel_count,pings_per_vessel
str,str,f64,f64,f64
"""count""","""22""",22.0,22.0,22.0
"""null_count""","""0""",0.0,0.0,0.0
"""mean""","""2015-11-15 21:49:05.454000""",237.409091,246.818182,71902.541558
"""std""",null,73.487063,75.187788,39790.523814
"""min""","""2015-01-01""",140.0,148.0,7512.459459
"""25%""","""2015-06-01""",168.0,175.0,39634.404255
"""50%""","""2015-12-01""",235.0,243.0,69020.938272
"""75%""","""2016-05-01""",297.0,311.0,105644.758842
"""max""","""2016-10-01""",356.0,367.0,138855.389049
